In [1]:
!pip uninstall -y ezsmdeploy

Found existing installation: ezsmdeploy 0.1.5
Uninstalling ezsmdeploy-0.1.5:
  Successfully uninstalled ezsmdeploy-0.1.5


## Train a model locally or remote

In [2]:
from sklearn import datasets, svm
digits = datasets.load_digits()
clf = svm.SVC(gamma=0.001, C=100.,probability=True)
clf.fit(digits.data[:-1], digits.target[:-1])
clf.predict(digits.data[-1:])

array([8])

### Save model file

In [3]:
from joblib import dump
dump(clf, 'model.joblib')

['model.joblib']

## Step 1 : Write a model transform script

#### Make sure you have a ...

- "load_model" function
    - input args are model path
    - returns loaded model object
    - model name is the same as what you saved the model file as (see above step)
<br><br>
- "predict" function
    - input args are the loaded model object and a payload
    - returns the result of model.predict
    - make sure you format it as a single (or multiple) string return inside a list for real time (for mini batch)
    - from a client, a list  or string or np.array that is sent for prediction is interpreted as bytes. Do what you have to for converting back to list or string or np.array
    - return the error for debugging


In [4]:
%%writefile modelscript_sklearn.py
import sklearn
from joblib import load
import numpy as np
import os

#Return loaded model
def load_model(modelpath):
    print(modelpath)
    clf = load(os.path.join(modelpath,'model.joblib'))
    print("loaded")
    return clf

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        # locally, payload may come in as an np.ndarray
        if type(payload)==np.ndarray:
            out = [str(model.predict(np.frombuffer(payload).reshape((1,64))))]
        # in remote / container based deployment, payload comes in as a stream of bytes
        else:
            out = [str(model.predict(np.frombuffer(payload[0]['body']).reshape((1,64))))]
    except Exception as e:
        out = [type(payload),str(e)] #useful for debugging!
    
    return out

Overwriting modelscript_sklearn.py


## Does this work locally? (not "_in a container locally_", but _actually_ in local)

In [5]:
from modelscript_sklearn import *
model = load_model('.')

.
loaded


In [6]:
predict(model,digits.data[-1:])[0]

'[8]'

### ok great! Now let's install ezsmdeploy

_[To Do]_: currently local; replace with pip version!

In [7]:
!pip install -e ./ --quiet 

In [8]:
import ezsmdeploy

#### If you have been running other inference containers in local mode, stop existing containers to avoid conflict

In [9]:
!docker container stop $(docker container ls -aq) >/dev/null

## Deploy locally

In [10]:
ez = ezsmdeploy.Deploy(model = ['model.joblib'], # if you intend to add models later, pass model as list, otherwise str
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.20.3','numpy','joblib'], #or pass in the path to requirements.txt
                  instance_type = 'local',
                  autoscale = True,
                  #framework = 'sklearn', # not required if you provide requirements
                  wait = True)

0:00:00.174121 | compressed model(s)
0:00:00.282179 | uploaded model tarball(s) ; check returned modelpath
0:00:00.283092 | added requirements file
0:00:00.284914 | added source file
0:00:00.286439 | added Dockerfile
0:00:00.288082 | added model_handler and docker utils
0:00:00.288149 | building docker container
0:00:56.402638 | built docker container
0:00:56.801667 | created model(s). Now deploying on local
∙●∙ Attaching to tmpbyk16cci_algo-1-v1kld_1
algo-1-v1kld_1  | Warning: Calling MMS with mxnet-model-server. Please move to multi-model-server.
●∙∙ algo-1-v1kld_1  | 2020-04-22 23:47:43,488 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-v1kld_1  | MMS Home: /usr/local/lib/python3.5/dist-packages
algo-1-v1kld_1  | Current directory: /
algo-1-v1kld_1  | Temp directory: /tmp
algo-1-v1kld_1  | Number of GPUs: 0
algo-1-v1kld_1  | Number of CPUs: 32
algo-1-v1kld_1  | Max heap size: 27305 M
algo-1-v1kld_1  | Python executable: /usr/bin/python3
algo-1-v1kld_1  | Config file: /etc/s

algo-1-v1kld_1  | 2020-04-22 23:47:43,763 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /tmp/.mms.sock.9000.
algo-1-v1kld_1  | 2020-04-22 23:47:43,764 [WARN ] pool-2-thread-1 com.amazonaws.ml.mms.metrics.MetricCollector - worker pid is not available yet.
algo-1-v1kld_1  | 2020-04-22 23:47:43,765 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /tmp/.mms.sock.9000.
algo-1-v1kld_1  | 2020-04-22 23:47:43,766 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /tmp/.mms.sock.9000.
algo-1-v1kld_1  | 2020-04-22 23:47:43,768 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /tmp/.mms.sock.9000.
algo-1-v1kld_1  | 2020-04-22 23:47:43,770 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /tmp/.mms.sock.9000.
algo-1-v1kld_1  | 2020-04-22 23:47:43,771 [INFO ] W-9000-model-stdout com.amazo

●∙∙ algo-1-v1kld_1  | 2020-04-22 23:47:44,718 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - loaded model!
algo-1-v1kld_1  | 2020-04-22 23:47:44,751 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-model-13
algo-1-v1kld_1  | 2020-04-22 23:47:44,751 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-model-29
algo-1-v1kld_1  | 2020-04-22 23:47:44,752 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 913
algo-1-v1kld_1  | 2020-04-22 23:47:44,752 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Model model loaded io_fd=0242acfffe1b0002-00000027-00000008-14e16a68e28787c6-36ef5e4e
algo-1-v1kld_1  | 2020-04-22 23:47:44,752 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - /.sagemaker/mms/models/model
algo-1-v1kld_1  | 2020-04-22 23:47:44,752 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - 

## Test containerized version locally

In [11]:
out = ez.predictor.predict(digits.data[-1:].tobytes()).decode()
out

algo-1-v1kld_1  | 2020-04-22 23:48:16,175 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 1
algo-1-v1kld_1  | 2020-04-22 23:48:16,176 [INFO ] W-9000-model ACCESS_LOG - /172.27.0.1:54188 "POST /invocations HTTP/1.1" 200 5


'[8]'

In [15]:
!docker container stop $(docker container ls -aq) >/dev/null

tmpbyk16cci_algo-1-v1kld_1 exited with code 137
Aborting on container exit...


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpbyk16cci/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 137



## Deploy on SageMaker

In [13]:
ezonsm = ezsmdeploy.Deploy(model = ['model.joblib','model.joblib'], # example of multimodel endpoint. 
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.20.3','numpy','joblib'])

0:00:00.343681 | compressed model(s)
0:00:00.774223 | uploaded model tarball(s) ; check returned modelpath
0:00:00.775314 | added requirements file
0:00:00.777319 | added source file
0:00:00.778630 | added Dockerfile
0:00:00.780260 | added model_handler and docker utils
0:00:00.780350 | building docker container
0:00:32.592506 | built docker container
0:00:32.925355 | created model(s). Now deploying on ml.m5.xlarge
0:07:05.373108 | deployed model
0:07:05.373703 | estimated cost is $0.3 per hour
0:07:05.374069 | Done! ✔ 


In [14]:
out = ezonsm.predictor.predict(digits.data[-1:].tobytes(),target_model='model1.tar.gz').decode() 
out

'[8]'

In [16]:
ezonsm.test(input_data=digits.data[-1:].tobytes(), target_model='model1.tar.gz',usercount=20,hatchrate=10,timeoutsecs=10)

0:00:00.001576 | Starting test with Locust
0:00:12.509494 | Done! Please see the src folder for locuststats* files


In [20]:
import pandas as pd

pd.read_csv('src/locuststats_stats.csv')

,Type,Name,# requests,# failures,Median response time,Average response time,Min response time,Max response time,Average Content Size,Requests/s,...,75%,80%,90%,95%,98%,99%,99.9%,99.99%,99.999,100%
0,sagemaker,predict,1731,1,13,27,10,7515,0,13.54,...,15,15,16,19,23,72,7500,7500,7500,7500
1,None,Aggregated,1731,1,13,27,10,7515,0,13.54,...,15,15,16,19,23,72,7500,7500,7500,7500


In [ ]:
ezonsm.predictor.delete_endpoint()